Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [9]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [10]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [7]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy + 2 opt 

In [13]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from icecream import ic
import random
import heapq

logging.basicConfig(level=logging.DEBUG)

# Load the cities dataset

# Initialize distance matrix
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

# Fill distance matrix using geodesic distance
for c1, c2 in combinations(CITIES.itertuples(), 2):
    distance = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = distance

# Construct graph as adjacency list with distance constraints
graph = {i: [] for i in range(len(CITIES))}
for c1, c2 in combinations(CITIES.itertuples(), 2):
    distance = DIST_MATRIX[c1.Index, c2.Index]
    if distance < 327:
        graph[c1.Index].append((c2.Index, distance))
        graph[c2.Index].append((c1.Index, distance))

# Implement Dijkstra's algorithm
def dijkstra(graph, start, end):
    queue = [(0, start)]
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous_nodes = {node: None for node in graph}
    
    while queue:
        current_distance, current_node = heapq.heappop(queue)
        
        if current_node == end:
            # Reconstruct the path from end to start
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = previous_nodes[current_node]
            return path[::-1], distances[end]
        
        if current_distance > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(queue, (distance, neighbor))
    
    return None, float('inf')  # Return None if no path exists

# Implement A* algorithm
def a_star(graph, start, end, positions):
    queue = [(0, start)]
    g_scores = {node: float('inf') for node in graph}
    f_scores = {node: float('inf') for node in graph}
    g_scores[start] = 0
    f_scores[start] = geodesic(positions[start], positions[end]).km
    previous_nodes = {node: None for node in graph}
    
    while queue:
        current_f_score, current_node = heapq.heappop(queue)
        
        if current_node == end:
            # Reconstruct the path from end to start
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = previous_nodes[current_node]
            return path[::-1], g_scores[end]
        
        for neighbor, weight in graph[current_node]:
            tentative_g_score = g_scores[current_node] + weight
            if tentative_g_score < g_scores[neighbor]:
                g_scores[neighbor] = tentative_g_score
                f_scores[neighbor] = tentative_g_score + geodesic(positions[neighbor], positions[end]).km
                previous_nodes[neighbor] = current_node
                heapq.heappush(queue, (f_scores[neighbor], neighbor))
    
    return None, float('inf')  # Return None if no path exists

# Prepare the city positions for A* heuristic
positions = {i: (CITIES.loc[i, 'lat'], CITIES.loc[i, 'lon']) for i in range(len(CITIES))}

# Select 10 random start and end cities
random.seed(42)
city_indices = list(range(len(CITIES)))
random_pairs = random.sample(list(combinations(city_indices, 2)), 10)

# Compare the algorithms
results = []

for start, end in random_pairs:
    # Run Dijkstra's algorithm
    dijkstra_path_result, dijkstra_distance = dijkstra(graph, start, end)
    
    # Run A* algorithm
    a_star_path_result, a_star_distance = a_star(graph, start, end, positions)
    
    results.append({
        "start": CITIES.loc[start, 'name'],
        "end": CITIES.loc[end, 'name'],
        "dijkstra_path": dijkstra_path_result,
        "dijkstra_distance": dijkstra_distance,
        "a_star_path": a_star_path_result,
        "a_star_distance": a_star_distance,
    })

# Output results
results_df = pd.DataFrame(results)
ic(results_df)




ic| results_df:                    start       end        dijkstra_path  dijkstra_distance  \
                0                Bolzano    Modena              [5, 19]         208.334929   
                1                 Andria   Catania       [1, 38, 17, 8]         495.616599   
                2  Giugliano in Campania     Terni             [14, 39]         222.669771   
                3                  Forlì    Trento             [12, 40]         218.743390   
                4                 Foggia     Prato         [11, 26, 29]         454.182608   
                5                Brescia  Syracuse  [6, 26, 14, 17, 37]        1058.260458   
                6                Bologna    Trento              [4, 40]         176.443036   
                7                Bologna   Ferrara               [4, 9]          43.427286   
                8                Pescara  Piacenza          [27, 4, 28]         469.807160   
                9                 Andria    Novara      [1, 

,start,end,dijkstra_path,dijkstra_distance,a_star_path,a_star_distance
0,Bolzano,Modena,"[5, 19]",208.334929,"[5, 19]",208.334929
1,Andria,Catania,"[1, 38, 17, 8]",495.616599,"[1, 38, 17, 8]",495.616599
2,Giugliano in Campania,Terni,"[14, 39]",222.669771,"[14, 39]",222.669771
3,Forlì,Trento,"[12, 40]",218.743390,"[12, 40]",218.743390
4,Foggia,Prato,"[11, 26, 29]",454.182608,"[11, 26, 29]",454.182608
5,Brescia,Syracuse,"[6, 26, 14, 17, 37]",1058.260458,"[6, 26, 14, 17, 37]",1058.260458
6,Bologna,Trento,"[4, 40]",176.443036,"[4, 40]",176.443036
7,Bologna,Ferrara,"[4, 9]",43.427286,"[4, 9]",43.427286
8,Pescara,Piacenza,"[27, 4, 28]",469.807160,"[27, 4, 28]",469.807160
9,Andria,Novara,"[1, 27, 29, 22]",781.059431,"[1, 27, 29, 22]",781.059431


## Greedy + evolutionary
